In [ ]:
from survivors.ensemble import ParallelBootstrapCRAID

class PBCRAID(ParallelBootstrapCRAID):
    def tolerance_find_best(self, ens_metric_name="bic"):
        self.ens_metric_name = ens_metric_name

        self.is_ci = self.ens_metric_name.upper().find("CI") >= 0
        self.is_ibs = self.ens_metric_name.upper().find("IBS") >= 0
        self.is_auprc = self.ens_metric_name.upper().find("AUPRC") >= 0
        self.is_iauc = self.ens_metric_name.upper().find("IAUC") >= 0

        ens_metr_arr = np.zeros(self.n_estimators)

        self.prepare_for_tolerance()
        for i in range(self.n_estimators):
            self.tolerance_iter = i
            self.predict_by_i(i)
            ens_metr_arr[i] = self.score_oob()

        descend_metr = self.ens_metric_name in metr.DESCEND_METRICS
        if descend_metr:
            best_index = np.argmin(ens_metr_arr)
        else:
            best_index = np.argmax(ens_metr_arr)
        self.select_model(0, best_index + 1)
        print(ens_metr_arr)
        print(f"fitted: {len(self.models)} models.")
        return ens_metr_arr
        
        
param_bstr = {'aggreg_func': 'mean',
             'categ': categ, 'criterion': 'logrank', 
             'depth': 5, 'ens_metric_name': 'CI', 'l_reg': 0, 'leaf_model': 
             'base_zero_after', 'max_features': 0.3, 'min_samples_leaf': 0.01, 'n_estimators': 50, 
             'n_jobs': 50, 'size_sample': 0.6, 'weighted_tree': False}


bstr = PBCRAID(**param_bstr)
bstr.fit(X_tr, y_tr)
ens_metr_arr = bstr.tolerance_find_best(param_bstr["ens_metric_name"])

In [ ]:
axis_font = {'fontname':'Times New Roman', 'size':'16'}
sns.set_context(rc={'lines.linewidth':2})

plt.subplots(figsize=(10, 6))
plt.plot(range(1, ens_metr_arr.shape[0]+1), ens_metr_arr, 'o-', label="Качество на OOB выборке")
plt.vlines(x=2, ymin=ens_metr_arr.min(), ymax=ens_metr_arr.max(), color="green", linewidth=3, label="Размер=2 по итеративному алгоритму")
plt.vlines(x=31, ymin=ens_metr_arr.min(), ymax=ens_metr_arr.max(), color="red", linewidth=3, label="Размер=31 по толерантному алгоритму")
plt.ylabel("Concordance index", **axis_font)
plt.xlabel("Размер ансамбля", **axis_font)
plt.legend(prop={'size': 16, 'family':'Times New Roman'})